In [1]:
# package imports
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure
from ase.visualize import view
import numpy as np
from datetime import datetime

# class imports
from vdW_structures.vdW_structure import VdWStructure
from vdW_structures.vdW_heterostructure import VdWHeterostructureGenerator
from vdW_structures.layer_shifter import LayerShifter
from vdW_structures.unique_structures import UniqueStructureGetter

## Input Bulk Structures

In [2]:
bise2 = Structure.from_file('original_structures/BiSe2.vasp')
bi2se3 = Structure.from_file('original_structures/Bi2Se3.vasp')
bi3se4 = Structure.from_file('original_structures/Bi3Se4.vasp')
bi4se3 = Structure.from_file('original_structures/Bi4Se3.vasp')
test = bi3se4

## Generate Starting vdW Subunits for BiSe2, Bi2Se3, Bi3Se4

In [3]:
vdW_bi2se3 = VdWStructure(bi2se3, minimum_vdW_gap=2.4)
vdW_bi3se4 = VdWStructure(bi3se4, minimum_vdW_gap=2.4)
vdW_bise2 = VdWStructure(bise2, minimum_vdW_gap=1.9)

# Note: the vdW_bise2 structure has a smaller minimum vdW gap than vdW_bi2se3 and vdW_bi3se4, which could cause issues
# To make this VdWStructure commensurate, create a vdW_gap in bise2 that is > 2.4

#This can be done two ways (equivalent for this system):

# Add vacuum to vdW_bise2. This works because bise2 is a single vdW unit. 
vdW_bise2_first = VdWStructure(vdW_bise2.set_vacuum(vacuum=2.41).structure, minimum_vdW_gap=2.4)
print(vdW_bise2_first.vdW_layers, vdW_bise2_first.vdW_spacings)

# Or, add padding to vdW layer 0 in vdW_bise2. This is how to change layer distances when multiple layers exist in a structure. 
vdW_bise2_second = VdWStructure(vdW_bise2.z_solve_shift_vdW_layers(layer_inds=0, 
                                                            solve_shift=2.41, 
                                                            adjust_z_cell=True).structure, 
                         minimum_vdW_gap=2.4)
print(vdW_bise2_second.vdW_layers, vdW_bise2_second.vdW_spacings)

[[1, 0, 2]] [2.41]
[[1, 0, 2]] [2.41]


In [4]:
# Look at the arrangement of vdW layers (by structure indices) and interlayer distances between the layers
print(vdW_bi2se3.vdW_layers, vdW_bi2se3.vdW_spacings)
print(vdW_bi3se4.vdW_layers, vdW_bi3se4.vdW_spacings)
print(vdW_bise2_second.vdW_layers, vdW_bise2_second.vdW_spacings)

[[10, 3, 9, 4, 14], [13, 5, 12, 0, 8], [7, 1, 6, 2, 11]] [2.9359044522460795, 2.9359044522460938, 2.9359044522460884]
[[9, 1, 12, 3, 15, 5, 14], [13, 4, 16, 6, 19, 8, 18], [17, 7, 20, 0, 11, 2, 10]] [3.0050793718446416, 3.0050793718446407, 3.0050793718446407]
[[1, 0, 2]] [2.41]


In [5]:
# View the vdW_bise2_second structure
view(vdW_bise2_second.structure.to_ase_atoms(), viewer='ngl')

In [6]:
# Choose which layers you'd like to include in the heterostructure construction
# For the current example, it's sufficient to use a single vdW layer from each structure

subunit_vdW_bi2se3_0 = vdW_bi2se3.extend_vdW_layers([0])
subunit_vdW_bi3se4_1 = vdW_bi3se4.extend_vdW_layers([1])
subunit_vdW_bise2_0 = vdW_bise2_second.extend_vdW_layers([0])

In [7]:
# Look at the Bi2Se3 subunit, with the appropriate vdW gap across the PBC
view(subunit_vdW_bi2se3_0.structure.to_ase_atoms(), viewer='ngl')

## Generate vdW Heterostructures

In [8]:
# This requires you to specify a ZSLGenerator structure, with the tolerances passed dictating the size(s) of vdW heterostructures generated

# This ZSL object yields four unique heterostructures for Bi2Se3 on Bi3Se4

bi2se3_bi3se4_zsl = ZSLGenerator(max_area_ratio_tol=0.5, 
                                 max_area=150, 
                                 max_length_tol=0.05, 
                                 max_angle_tol=0.05)

vsg = VdWHeterostructureGenerator()
start = datetime.now()
vdW_heterostructures_bi2se3_bi3se4 = vsg.generate_unique_vdW_heterostructures(film=subunit_vdW_bi2se3_0, 
                                                                              substrate=subunit_vdW_bi3se4_1, 
                                                                              zsl_generator=bi2se3_bi3se4_zsl, 
                                                                              n_chunks=4, 
                                                                              ltol=0.2, stol=0.3, angle_tol=5)
end = datetime.now()
print(end - start)
# Print the lengths of the unique heterostructures
s_vdW_heterostructures_bi2se3_bi3se4 = sorted(vdW_heterostructures_bi2se3_bi3se4, key=lambda obj: len(obj.structure))


print([len(heterostructure.structure) for heterostructure in s_vdW_heterostructures_bi2se3_bi3se4])


/Users/rym/miniconda3/lib/python3.9/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/rym/miniconda3/lib/python3.9/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


Unique structures: 2
Unique structures: 2
Unique structures: 4
Unique structures: 4
Unique structures: 4
0:00:12.435890
[12, 24, 84, 84]


In [10]:
# We'll use the first heterostructure returned, which has the fewest # of atoms
view(vdW_heterostructures_bi2se3_bi3se4[0].structure.to_ase_atoms(), viewer='ngl')

In [11]:
# Add a BiSe2 film to this substrate
bise2_bi2se3_bi3se4_zsl = ZSLGenerator(max_area_ratio_tol=0.5, 
                                 max_area=150, 
                                 max_length_tol=0.05, 
                                 max_angle_tol=0.05)

vdW_heterostructures_bise2_bi2se3_bi3se4 = vsg.generate_unique_vdW_heterostructures(film=subunit_vdW_bise2_0, 
                                                                              substrate=vdW_heterostructures_bi2se3_bi3se4[0], 
                                                                              zsl_generator=bise2_bi2se3_bi3se4_zsl)

# Print the lengths of the unique heterostructures
print([len(heterostructure.structure) for heterostructure in vdW_heterostructures_bise2_bi2se3_bi3se4])

/Users/rym/miniconda3/lib/python3.9/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/rym/miniconda3/lib/python3.9/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


Unique structures: 6
Unique structures: 6
Unique structures: 7
Unique structures: 7
Unique structures: 7
[15, 30, 30, 45, 45, 60, 105]


In [12]:
# And view the first heterostructure generated
view(vdW_heterostructures_bise2_bi2se3_bi3se4[0].structure.to_ase_atoms(), viewer='ngl')

In [13]:
# Generate a cubic supercell for easier viewing

from pymatgen.transformations.advanced_transformations import CubicSupercellTransformation
cst = CubicSupercellTransformation(force_90_degrees=True)

# Choose the heterostruture at index 3
index = 1
sc_heterostructure = VdWStructure(cst.apply_transformation(vdW_heterostructures_bise2_bi2se3_bi3se4[index].structure), 
                                            vdW_heterostructures_bise2_bi2se3_bi3se4[index].minimum_vdW_gap)

In [14]:
view(sc_heterostructure.structure.to_ase_atoms(), viewer='ngl')

In [15]:
len(sc_heterostructure.structure)

240

## Modifying the Heterostructure

### Shift a layer within the fixed box

In [16]:
# Can be performed for x, y , or z directions, but we'll shift relative to z because it's easiest to visualize

z_shifted_layer_1 = sc_heterostructure.z_shift_vdW_layers(layer_inds=1, 
                                                          shift=-1.0, 
                                                          coords_are_cartesian=True)
view(z_shifted_layer_1.structure.to_ase_atoms(), viewer='ngl')

### Increase interlayer distance, preserving other vdW spacings

In [17]:
# Increase a layer and increase box to preserve vdW spacing
increased_gap_layer_2 = sc_heterostructure.z_solve_shift_vdW_layers(layer_inds=[2, 0], 
                                                                    solve_shift=[2.8, 3.4], 
                                                                    adjust_z_cell=True)
view(increased_gap_layer_2.structure.to_ase_atoms(), viewer='ngl')

In [18]:
print(increased_gap_layer_2.vdW_spacings)

[3.400000000000006, 2.9359044522460724, 2.7999999999999936]


### Change the layering, so that it is Bi3Se4 - Bi2Se3 - BiSe2 - Bi2Se3

In [19]:
modified_layering = sc_heterostructure.extend_vdW_layers([0, 1, 2, 1])
view(modified_layering.structure.to_ase_atoms(), viewer='ngl')

In [20]:
print(modified_layering.vdW_spacings)

[3.005079371844629, 2.935904452246076, 2.4099999999999824, 2.93590445224606]
